In [1]:
import geopandas as gpd
import pandas as pd
import os,re,json
import numpy as np
from p_tqdm import p_map

In [2]:

# pip install mbutil

In [21]:
# lets generate the overview pictures 
#
floc = "tmp/county.geojson"
gpd.read_file('data/CountiesUA.geojson')[['OBJECTID', 'CTYUA20CD', 'CTYUA20NM','LONG', 'LAT', 'geometry']].to_file(floc, driver="GeoJSON")

os.system('tippecanoe -Z0 -z6 -f -o tiles/countries.mbtiles'+ floc)


tippecanoe: tiles/countries.mbtilestmp/county.geojson: unable to open database file


256

In [30]:
 gpd.read_file('data/centroids2001.geojson').columns
    


Index(['OBJECTID', 'OA01CD', 'OA01CDOLD', 'geometry'], dtype='object')

In [38]:
year = '01'

base  = gpd.read_file('data/OA2001.geojson')[['OA01CD', 'geometry']]
centroids = gpd.read_file('data/centroids2001.geojson')[['OA01CD','geometry']]

base.to_file("tmp/OA%s.geojson"%year, driver="GeoJSON")
centroids.to_file("tmp/centroids%s.geojson"%year, driver="GeoJSON")

start = 6
end = 12
# cant have centroids being grouped, but fine for area shapes
os.system('tippecanoe -Z%d -z%d -f -r1 -o tiles/centroids%s.mbtiles  ./tmp/centroids%s.geojson'%(start,end,year,year))
os.system('tippecanoe -Z%d -z%d -f -o tiles/outputarea%s.mbtiles --drop-densest-as-needed --generate-id ./tmp/OA%s.geojson'%(start,end,year,year))

os.system('tile-join -o ./tiles/area_centroids_6_9.mbtiles -f ./tiles/centroids%s.mbtiles ./tiles/outputarea%s.mbtiles'%(year,year))

# --generate-id
# --coalesce-densest-as-needed

# tippecanoe -o tiles/test.mbtiles -Z9 -z10 -f ./data/tile_OA11.geojson ./data/tile_centroids11.geojson


# gdf_base = gdf_base.append(df,ignore_index=True)
# gdf_base .to_file("myfile.gpkg", layer="layer1", driver="GPKG")

For layer 0, using name "centroids01"
175434 features, 3178644 bytes of geometry, 16 bytes of separate metadata, 1929782 bytes of string pool
tile 6/31/20 size is 570318 with detail 12, >500000    
tile 6/31/21 size is 636976 with detail 12, >500000    
tile 6/31/20 size is 520253 with detail 11, >500000    
tile 6/31/21 size is 579801 with detail 11, >500000    
tile 6/31/21 size is 504547 with detail 10, >500000    
tile 7/63/42 size is 533143 with detail 12, >500000    
  99.9%  12/2027/1295  
For layer 0, using name "OA01"
175434 features, 50384833 bytes of geometry, 1041 bytes of separate metadata, 1929782 bytes of string pool
tile 6/32/21 size is 533433 with detail 12, >500000    
Going to try keeping the sparsest 84.36% of the features to make it fit
tile 6/31/20 size is 1590858 with detail 12, >500000    
Going to try keeping the sparsest 28.29% of the features to make it fit
tile 6/31/21 size is 1658288 with detail 12, >500000    
Going to try keeping the sparsest 27.14% of th

0

In [39]:
# base  = gpd.read_file('data/OA2011.geojson')[['OA01CD', 'LAD01CD',
#        'geometry']]
# centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

# base.to_file("tmp/OA11.geojson", driver="GeoJSON")
# centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

# os.system('tippecanoe -Z6 -z9 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))




In [40]:
# merge
os.system('tile-join -o ./tiles/merged.mbtiles -f ./tiles/countries.mbtiles ./tiles/area_centroids_6_9.mbtiles --attribution=ONS')

0

In [41]:
# tippecanoe -z3 -o countries-z3.mbtiles --coalesce-densest-as-needed ne_10m_admin_0_countries.geojson
# tippecanoe -zg -Z4 -o states-Z4.mbtiles --coalesce-densest-as-needed --extend-zooms-if-still-dropping ne_10m_admin_1_states_provinces.geojson
# tile-join -o states-countries.mbtiles countries-z3.mbtiles states-Z4.mbtiles

In [42]:
## convert to static server

os.system('rm -rf ./tiles/tileserver; mb-util --image_format=pbf ./tiles/merged.mbtiles ./tiles/tileserver')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % gzip -d -r -S .pbf % 2>&1')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % mv % %.pbf 2>&1')


DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:./tiles/merged.mbtiles --> ./tiles/tileserver
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:12 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:13 / 6464 tiles exported
DEBU

DEBUG:mbutil.util:flipping
INFO:mbutil.util:686 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:687 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:688 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:689 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:690 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:691 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:692 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:693 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:694 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:695 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:696 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:697 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:698 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:699 / 6464 tiles exported
DEBUG:mbutil.util:fl

INFO:mbutil.util:1473 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1474 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1475 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1476 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1477 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1478 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1479 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1480 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1481 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1482 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1483 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1484 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1485 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1486 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:m

DEBUG:mbutil.util:flipping
INFO:mbutil.util:2321 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2322 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2323 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2324 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2325 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2326 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2327 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2328 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2329 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2330 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2331 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2332 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2333 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2334 / 6464 tiles exported
DEBUG:

DEBUG:mbutil.util:flipping
INFO:mbutil.util:3206 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3207 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3208 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3209 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3210 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3211 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3212 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3213 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3214 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3215 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3216 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3217 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3218 / 6464 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3219 / 6464 tiles exported
DEBUG:

...

mv ./12/1998/1354 ./12/1998/1354.pbf
mv ./12/1998/1353 ./12/1998/1353.pbf
mv ./12/1998/1336 ./12/1998/1336.pbf
mv ./12/1998/1331 ./12/1998/1331.pbf
mv ./12/1991/1381 ./12/1991/1381.pbf
mv ./12/1991/1386 ./12/1991/1386.pbf
mv ./12/1991/1380 ./12/1991/1380.pbf
mv ./12/1991/1356 ./12/1991/1356.pbf
mv ./12/1991/1358 ./12/1991/1358.pbf
mv ./12/1991/1360 ./12/1991/1360.pbf
mv ./12/1991/1359 ./12/1991/1359.pbf
mv ./12/1991/1357 ./12/1991/1357.pbf
mv ./12/1991/1385 ./12/1991/1385.pbf
mv ./12/1991/1382 ./12/1991/1382.pbf
mv ./12/1991/1378 ./12/1991/1378.pbf
mv ./12/1991/1379 ./12/1991/1379.pbf
mv ./12/1991/1383 ./12/1991/1383.pbf
mv ./12/1991/1384 ./12/1991/1384.pbf
mv ./12/1991/1352 ./12/1991/1352.pbf
mv ./12/1991/1355 ./12/1991/1355.pbf
mv ./12/1991/1354 ./12/1991/1354.pbf
mv ./12/1991/1353 ./12/1991/1353.pbf
mv ./12/1996/1375 ./12/1996/1375.pbf
mv ./12/1996/1381 ./12/1996/1381.pbf
mv ./12/1996/1372 ./12/1996/1372.pbf
mv ./12/1996/1328 ./12/1996/1328.pbf
mv ./12/1996/1326 ./12/1996/1326.pbf
m

mv ./12/2034/1357 ./12/2034/1357.pbf
mv ./12/2034/1368 ./12/2034/1368.pbf
mv ./12/2034/1313 ./12/2034/1313.pbf
mv ./12/2034/1314 ./12/2034/1314.pbf
mv ./12/2034/1322 ./12/2034/1322.pbf
mv ./12/2034/1325 ./12/2034/1325.pbf
mv ./12/2034/1371 ./12/2034/1371.pbf
mv ./12/2034/1349 ./12/2034/1349.pbf
mv ./12/2034/1376 ./12/2034/1376.pbf
mv ./12/2034/1340 ./12/2034/1340.pbf
mv ./12/2034/1378 ./12/2034/1378.pbf
mv ./12/2034/1347 ./12/2034/1347.pbf
mv ./12/2034/1324 ./12/2034/1324.pbf
mv ./12/2034/1323 ./12/2034/1323.pbf
mv ./12/2034/1315 ./12/2034/1315.pbf
mv ./12/2034/1312 ./12/2034/1312.pbf
mv ./12/2034/1346 ./12/2034/1346.pbf
mv ./12/2034/1341 ./12/2034/1341.pbf
mv ./12/2034/1348 ./12/2034/1348.pbf
mv ./12/2034/1377 ./12/2034/1377.pbf
mv ./12/2034/1370 ./12/2034/1370.pbf
mv ./12/2034/1352 ./12/2034/1352.pbf
mv ./12/2034/1355 ./12/2034/1355.pbf
mv ./12/2034/1363 ./12/2034/1363.pbf
mv ./12/2034/1364 ./12/2034/1364.pbf
mv ./12/2034/1330 ./12/2034/1330.pbf
mv ./12/2034/1308 ./12/2034/1308.pbf
m

mv ./12/2066/1338 ./12/2066/1338.pbf
mv ./12/2059/1372 ./12/2059/1372.pbf
mv ./12/2059/1344 ./12/2059/1344.pbf
mv ./12/2059/1343 ./12/2059/1343.pbf
mv ./12/2059/1342 ./12/2059/1342.pbf
mv ./12/2059/1345 ./12/2059/1345.pbf
mv ./12/2059/1334 ./12/2059/1334.pbf
mv ./12/2059/1356 ./12/2059/1356.pbf
mv ./12/2059/1369 ./12/2059/1369.pbf
mv ./12/2059/1351 ./12/2059/1351.pbf
mv ./12/2059/1367 ./12/2059/1367.pbf
mv ./12/2059/1335 ./12/2059/1335.pbf
mv ./12/2059/1366 ./12/2059/1366.pbf
mv ./12/2059/1350 ./12/2059/1350.pbf
mv ./12/2059/1357 ./12/2059/1357.pbf
mv ./12/2059/1368 ./12/2059/1368.pbf
mv ./12/2059/1371 ./12/2059/1371.pbf
mv ./12/2059/1349 ./12/2059/1349.pbf
mv ./12/2059/1340 ./12/2059/1340.pbf
mv ./12/2059/1347 ./12/2059/1347.pbf
mv ./12/2059/1346 ./12/2059/1346.pbf
mv ./12/2059/1341 ./12/2059/1341.pbf
mv ./12/2059/1348 ./12/2059/1348.pbf
mv ./12/2059/1370 ./12/2059/1370.pbf
mv ./12/2059/1352 ./12/2059/1352.pbf
mv ./12/2059/1355 ./12/2059/1355.pbf
mv ./12/2059/1364 ./12/2059/1364.pbf
m

mv ./12/2032/1367 ./12/2032/1367.pbf
mv ./12/2032/1358 ./12/2032/1358.pbf
mv ./12/2032/1360 ./12/2032/1360.pbf
mv ./12/2032/1303 ./12/2032/1303.pbf
mv ./12/2032/1304 ./12/2032/1304.pbf
mv ./12/2032/1332 ./12/2032/1332.pbf
mv ./12/2032/1335 ./12/2032/1335.pbf
mv ./12/2032/1361 ./12/2032/1361.pbf
mv ./12/2032/1366 ./12/2032/1366.pbf
mv ./12/2032/1359 ./12/2032/1359.pbf
mv ./12/2032/1350 ./12/2032/1350.pbf
mv ./12/2032/1357 ./12/2032/1357.pbf
mv ./12/2032/1368 ./12/2032/1368.pbf
mv ./12/2032/1295 ./12/2032/1295.pbf
mv ./12/2032/1313 ./12/2032/1313.pbf
mv ./12/2032/1314 ./12/2032/1314.pbf
mv ./12/2032/1322 ./12/2032/1322.pbf
mv ./12/2032/1325 ./12/2032/1325.pbf
mv ./12/2032/1371 ./12/2032/1371.pbf
mv ./12/2032/1349 ./12/2032/1349.pbf
mv ./12/2032/1376 ./12/2032/1376.pbf
mv ./12/2032/1340 ./12/2032/1340.pbf
mv ./12/2032/1378 ./12/2032/1378.pbf
mv ./12/2032/1347 ./12/2032/1347.pbf
mv ./12/2032/1324 ./12/2032/1324.pbf
mv ./12/2032/1323 ./12/2032/1323.pbf
mv ./12/2032/1315 ./12/2032/1315.pbf
m

0

In [43]:
os.system("git add -A; git commit -m 'recompute tiles'; git push")

[main ec0671b6] recompute tiles
 1663 files changed, 370215 insertions(+), 114248 deletions(-)
 delete mode 100644 tiles/tileserver/10/511/340.pbf
 rewrite tiles/tileserver/6/31/20.pbf (91%)
 create mode 100644 tiles/tileserver/6/31/21.pbf
 delete mode 100644 tiles/tileserver/7/63/41.pbf
 delete mode 100644 tiles/tileserver/7/63/42.pbf
 delete mode 100644 tiles/tileserver/8/126/82.pbf
 delete mode 100644 tiles/tileserver/8/127/85.pbf


remote: warning: See http://git.io/iEPt8g for more information.        
remote: warning: File tiles/merged.mbtiles is 83.45 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: File tiles/area_centroids_6_9.mbtiles is 83.02 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: File tiles/outputarea01.mbtiles is 77.81 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
To https://github.com/wolfiex/ONStileBuilder.git
   19cf177d..ec0671b6  main -> main


0